<a href="https://colab.research.google.com/github/TobyChen320/DS-Unit-2-Applied-Modeling/blob/master/Toby_Chen_DSPT6_Applied_Modeling_2_NBA_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install kaggle

In [0]:
# from google.colab import files
# files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"tobychendspt6","key":"72d63383e9d439abffa887f2adb12bdd"}'}

In [0]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [0]:
# !kaggle datasets download -d justinas/nba-players-data

401 - Unauthorized


In [0]:
# from zipfile import ZipFile
# file_name = "nba-players-data.zip"

# with ZipFile(file_name, 'r') as zip:
#   zip.extractall()
#   print('Done')

FileNotFoundError: ignored

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Data/all_seasons.csv')
df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,27,55,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97
1,1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,24,15,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97
2,2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,58,9,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97
3,3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,9,64,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97
4,4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,10,27,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97


In [0]:
%matplotlib inline
!pip install category_encoders==2.*
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import category_encoders as ce
import graphviz
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.tree import DecisionTreeClassifier

     |████████████████████████████████| 81kB 2.2MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
df['net_rating'].describe()

count    11145.000000
mean        -2.153899
std         12.150611
min       -200.000000
25%         -6.300000
50%         -1.300000
75%          3.200000
max        300.000000
Name: net_rating, dtype: float64

In [0]:
df = df.dropna(subset=['net_rating'])
df['Good'] = df['net_rating'] >= 1
df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Good
0,0,Dennis Rodman,CHI,36.0,198.12,99.790240,Southeastern Oklahoma State,USA,1986,2,27,55,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97,True
1,1,Dwayne Schintzius,LAC,28.0,215.90,117.933920,Florida,USA,1990,1,24,15,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97,True
2,2,Earl Cureton,TOR,39.0,205.74,95.254320,Detroit Mercy,USA,1979,3,58,9,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97,False
3,3,Ed O'Bannon,DAL,24.0,203.20,100.697424,UCLA,USA,1995,1,9,64,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97,False
4,4,Ed Pinckney,MIA,34.0,205.74,108.862080,Villanova,USA,1985,1,10,27,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97,False


In [0]:
y = df['Good']
y.nunique()

2

In [0]:
y.value_counts(normalize=True)

False    0.630238
True     0.369762
Name: Good, dtype: float64

In [0]:
df['player_name'].nunique()

2235

In [0]:
df['player_name'].unique()

array(['Dennis Rodman', 'Dwayne Schintzius', 'Earl Cureton', ...,
       'Max Strus', 'Norvel Pelle', 'Matt Thomas'], dtype=object)

In [0]:
df['player_name'] = df['player_name'].str.lower()
cardinality = df.select_dtypes(exclude='number').nunique()
high_cardinality = cardinality[cardinality > 50].index.tolist()
high_cardinality

['player_name', 'college', 'country', 'draft_number']

In [0]:
high_cardinality.remove('player_name')
high_cardinality

['college', 'country', 'draft_number']

In [0]:
df = df.drop(columns = high_cardinality)
df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,draft_year,draft_round,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,Good
0,0,dennis rodman,CHI,36.0,198.12,99.790240,1986,2,55,5.7,16.1,3.1,16.1,0.186,0.323,0.100,0.479,0.113,1996-97,True
1,1,dwayne schintzius,LAC,28.0,215.90,117.933920,1990,1,15,2.3,1.5,0.3,12.3,0.078,0.151,0.175,0.430,0.048,1996-97,True
2,2,earl cureton,TOR,39.0,205.74,95.254320,1979,3,9,0.8,1.0,0.4,-2.1,0.105,0.102,0.103,0.376,0.148,1996-97,False
3,3,ed o'bannon,DAL,24.0,203.20,100.697424,1995,1,64,3.7,2.3,0.6,-8.7,0.060,0.149,0.167,0.399,0.077,1996-97,False
4,4,ed pinckney,MIA,34.0,205.74,108.862080,1985,1,27,2.4,2.4,0.2,-11.2,0.109,0.179,0.127,0.611,0.040,1996-97,False


In [0]:
# train = df[(df['season'] >= '1996') & (df['season'] <= '2004')]
# val = df[(df['season'] >= '2005') & (df['season'] <= '2012')]
# test = df[(df['season'] >= '2013') & (df['season'] <= '2020')]
# train.shape, val.shape, test.shape

In [0]:
# Maybe random split would be better.
target = 'Good'
test = df[target]
train, val = train_test_split(df, train_size=0.80, test_size=0.20, random_state=42)

In [0]:
features = train.columns.drop([target, 'season', 'net_rating', 'Unnamed: 0'])
features

Index(['player_name', 'team_abbreviation', 'age', 'player_height',
       'player_weight', 'draft_year', 'draft_round', 'gp', 'pts', 'reb', 'ast',
       'oreb_pct', 'dreb_pct', 'usg_pct', 'ts_pct', 'ast_pct'],
      dtype='object')

In [0]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((8916, 16), (8916,), (2229, 16), (2229,))

In [0]:
train.groupby('player_name')[target].count().sort_values(ascending=False)

player_name
vince carter         19
paul pierce          17
tim duncan           17
chauncey billups     17
metta world peace    16
                     ..
david wear            1
david vanterpool      1
david noel            1
luke zeller           1
zylan cheatham        1
Name: Good, Length: 2124, dtype: int64

In [0]:
train_wrangled = train.groupby('player_name')[target].any().reset_index()
train_wrangled.head(510)

,player_name,Good
0,a.c. green,True
1,a.j. bramlett,False
2,a.j. guyton,False
3,aaron brooks,True
4,aaron gordon,False
...,...,...
505,demetris nichols,True
506,demetrius jackson,True
507,deng adel,False
508,dennis rodman,False


In [0]:
train_wrangled.shape

(2124, 2)

In [0]:
train_wrangled[target].value_counts(normalize=True)

True     0.594633
False    0.405367
Name: Good, dtype: float64

In [0]:
players = train_wrangled[train_wrangled['Good']==True]
players

,player_name,Good
0,a.c. green,True
3,aaron brooks,True
5,aaron gray,True
6,aaron harrison,True
7,aaron holiday,True
...,...,...
2115,zeljko rebraca,True
2116,zendon hamilton,True
2117,zhaire smith,True
2119,zion williamson,True


In [0]:
# pipeline = make_pipeline(
#     ce.ordinal.OrdinalEncoder(),
#     DecisionTreeClassifier()
# )
# pipeline.fit(X_train, y_train)
# print(f'Validation accuracy: {pipeline.score(X_val, y_val)}')

Validation accuracy: 0.616868550919695


In [0]:
# majority_class = y_train.mode()[0]
# y_pred = [majority_class] * len(y_val)
# accuracy_score(y_val, y_pred)

0.6289816061013908

In [0]:
# y_pred = pipeline.predict_proba(X_val)[:, 1]
# roc_auc_score(y_val, y_pred)

0.5972423226794681